In [ ]:
!pip install pandasai
!pip install streamlit
!pip install requests
!pip install -q -U google-generativeai

In [ ]:
pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40702 sha256=a65295219b41e4a82428a41d5fe8e5b7046f72ebfba55b830fad65cfde0a5557
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [ ]:
%%writefile UI.py

import streamlit as st
import requests
import pandas as pd
import google.generativeai as genai
from datetime import date, timedelta
from pandasai import SmartDataframe
from pandasai.llm import GooglePalm
import matplotlib.pyplot as plt
from fpdf import FPDF
import base64

st.set_page_config(layout="wide")

# Define the sidebar content
with st.sidebar:
    logo_url = "https://i.postimg.cc/5yLPdYWd/GURU-Finance.jpg"  # Replace with your actual logo URL
    st.markdown(
        f"""
        <style>
        .sidebar-logo-container {{
            display: flex;
            justify-content: center;
            align-items: center;
            height: 200px;  /* Adjust as needed */
        }}
        .sidebar-logo {{
            width: 270px;
        }}
        </style>
        <div class="sidebar-logo-container">
            <img src="{logo_url}" class="sidebar-logo">
        </div>
        """,
        unsafe_allow_html=True
    )

#logo_url = "https://i.postimg.cc/hjwywcWR/Black-White-Minimalist-Aesthetic-Initials-Font-Logo.jpg"


# Set sidebar background color
st.markdown("""
<style>
    [data-testid=stSidebar] {
        background-color: #FBEDBE;
    }
</style>
""", unsafe_allow_html=True)

# Define the sidebar content
with st.sidebar:
    pass  # Add any sidebar content here if needed

# Set the page background image
page_bg_img = """
<style>
[data-testid="stAppViewContainer"] > .main {
    background-image: url("https://i.postimg.cc/RZ6F5THr/Background-2.jpg");
    background-size: cover;
    background-position: center center;
    background-repeat: no-repeat;
    background-attachment: local;
}
[data-testid="stHeader"] {
    background: rgba(0,0,0,0);
}
</style>
"""

# Apply the page background image
st.markdown(page_bg_img, unsafe_allow_html=True)

# Mapping of original column names to display names
column_display_names = {
        'totalAssets': 'Total Assets',
        'totalLiabilities': 'Total Liabilities',
        'totalEquity': 'Total Equity',
        'totalRevenueAccum': 'Total Revenue Accumulated',
        'netProfitAccum': 'Net Profit Accumulated',
    }
# Reverse the mapping for easy lookup
display_to_original = {v: k for k, v in column_display_names.items()}
#st.sidebar.image(logo_url)
symbol = st.sidebar.text_input('Enter Symbol:', placeholder='Example : VIBHA')
symbol = symbol.upper()
start_year = st.sidebar.number_input('Start Year:', min_value=2000, max_value=2024, value=2022)
start_quarter = st.sidebar.selectbox('Start Quarter:', [1, 2, 3, 4], index=0)
end_year = st.sidebar.number_input('End Year:', min_value=2000, max_value=2024, value=2024)
end_quarter = st.sidebar.selectbox('End Quarter:', [1, 2, 3, 4], index=0)
graph_type = st.sidebar.selectbox('Select Graph Type:', ['bar', 'line', 'scatter'], index=0)
x_column = st.sidebar.selectbox('Select X-axis column:', ['Year_Quarter', 'year'], index=0)
y_column_display = st.sidebar.selectbox('Select Y-axis column:', list(display_to_original.keys()), index=0)
y_column = display_to_original[y_column_display]
aggregation_type = st.sidebar.selectbox('Select Aggregation Type:', ['sum', 'average'], index=0)
auto_size = st.sidebar.checkbox('Auto-size graph', value=True)
color = st.sidebar.color_picker('Pick a color:', '#e89d1c')

# Function to fetch financial data from API
def get_fin_data(symbol, start_year, start_quarter, end_year, end_quarter):
    url = "https://www.setsmart.com/api/listed-company-api/financial-data-and-ratio-by-symbol"
    params = {
        "symbol": symbol,
        "startYear": start_year,
        "startQuarter": start_quarter,
        "endYear": end_year,
        "endQuarter": end_quarter
    }
    headers = {
        "api-key": "Replace This With Your API Key"
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'data' in data:
            df = pd.DataFrame(data['data'])
        else:
            df = pd.DataFrame(data)
        return df
    else:
        return pd.DataFrame()

# Function to fetch quote data from API
def get_quote_data(symbol, start_year, start_quarter, end_year, end_quarter):
    startDate = date(start_year, (start_quarter - 1) * 3 + 1, 1)
    endDate = date(end_year, end_quarter * 3 + 1, 1) - timedelta(days=1)
    url = "https://www.setsmart.com/api/listed-company-api/eod-price-by-symbol"
    params = {
        "symbol": symbol,
        "startDate": startDate,
        "endDate": endDate,
        "adjustedPriceFlag": 'Y'
    }
    headers = {
        "api-key": "Replace This With Your API Key"
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'data' in data:
            df = pd.DataFrame(data['data'])
        else:
            df = pd.DataFrame(data)
        return df
    else:
        return pd.DataFrame()

# Function to convert df to text with row
def convert_df_to_text(df):
    text_representation = ""
    for _, row in df.iterrows():
        text_representation += str(row) + '\n'
    return text_representation

# Function to connect gemini
def connect_gemini():
    GOOGLE_API_KEY = 'Replace This With Your API Key'
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-pro')
    return model

def create_quote_dict():
    quote_dict = {
        "Column Name": ["date","symbol","securityType","adjustedPriceFlag","prior","open","high","low","close","average","aomVolume","aomValue","trVolume","trValue","totalVolume","totalValue","pe","pbv","bvps","dividendYield","marketCap","volumeTurnover"],
        "Data Type": ["string","string","string","string","number","number","number","number","number","number","number","number","number","number","number","number","number","number","number","number","number","number"],
        "Description": [
            "Trading date","Security symbol","Security Type - CS = Common Stock - CSF = Foreign Stock - PS = Preferred Stock - PSF = Preferred Foreign Stock - W = Warrant - TSR = Transferable Subscription Rights - DWC = Derivative Call Warrants - DWP = Derivative Put Warrants - DR = Depository Receipts - ETF = Exchange Traded Fund - UT = Unit Trust","Adjusted price flag - 'Y' = Price and volume are adjusted information - 'N' = Price and volume are not adjusted information","Prior closing price (If there is no trades yesterday, it will carry latest closing price)","Opening price of the day","Highest price of the day","Lowest price of the day","Last executed price","Average price","Auto-matching volume","Auto-matching value","Trade report volume","Trade report value","Total trading volume","Total trading value","P/E ratio - If P/E is negative, this field will be -100000.00. - If P/E is not calculated, for example, a security posted with SP sign more than 3 months, this field will be null. - P/E is not calculated for PF&REITs, Infrastructure Fund, ETF and UnitTrust. This field will be null","P/BV ratio - If P/BV is negative, this field will be -100000.00. - If P/BV is not calculated, for example, a security posted with SP sign more than 3 months, this field will be null. - For PF&REITs, Infrastructure Fund, ETF and UnitTrust, this field means P/NAV","Book value per share - Unit: Baht per Share - for PF&REIT, Infrastructure Fund, ETF or Unit trust, it means NAV","This value will be null if dividend yield is not calculated, for example, - a security which has no cash dividend payment - a security posted with SP sign more than 3 months","- Unit: Baht - This field will be null if market cap. is not calculated, for example, a security posted with SP sign more than 3 months","This value will be null if volume turnover is not calculated, for example, - a security posted with SP sign more than 3 months"]
        }
    quote_data_dict_df = pd.DataFrame(quote_dict)
    quote_data_dict_text = convert_df_to_text(quote_data_dict_df)
    quote_data_dict_text = "df_quo DataFrame:\n" + quote_data_dict_text
    return quote_data_dict_text

def create_financial_dict():
    fin_dict = {
        "Column Name": ["symbol","year","quarter","financialStatementType","dateAsof","accountPeriod","totalAssets","totalLiabilities","paidupShareCapital","shareholderEquity","totalEquity","totalRevenueQuarter","totalRevenueAccum","totalExpensesQuarter","totalExpensesAccum","ebitQuarter","ebitAccum","netProfitQuarter","netProfitAccum","epsQuarter","epsAccum","operatingCashFlow","investingCashFlow","financingCashFlow","roe","roa","netProfitMarginQuarter","netProfitMarginAccum","de","fixedAssetTurnover","totalAssetTurnover"],
        "Data Type": ["string","string","string","string","string","string","number","number","number","number","number","number","number","number","number","number","number","number","number","number","number","number","number","number","number","number","number","number","number","number","number"],
        "Description": [
            "Security symbol","Year of financial statement"
            ,"Quarter of financial statement"
            ,"Financial statement type - C = Consolidate - E = Equity Method - U = Company Only one financial statement type will be sent out for one symbol. - If there are financial statement type 'C' and 'U', only 'C' will be sent out - If there are financial statement type 'E' and 'U', only 'E' will be sent out - Financial statement type 'U' will be sent out if there is no 'C' nor 'E'"
            ,"As of date of financial statement","Account Period - F = Fiscal Year - C = Calendar Year"
            ,"Total assets Unit: Thousand Baht","Total liabilities Unit: Thousand Baht","Paidup capital Unit: Thousand Baht"
            ,"Shareholders' equity Unit: Thousand Baht","Total equity Unit: Thousand Baht","Total revenue (value of that quarter) Unit: Thousand Baht"
            ,"Total revenue (accumulated value) Unit: Thousand Baht","Total expense (value of that quarter) Unit: Thousand Baht"
            ,"Total expense (accumulated value) Unit: Thousand Baht","EBIT (value of that quarter) Unit: Thousand Baht","EBIT (accumulated value) Unit: Thousand Baht"
            ,"Net profit (value of that quarter) Unit: Thousand Baht","Net profit (accumulated value) Unit: Thousand Baht","Earning per share (value of that quarter) Unit: Thousand Baht"
            ,"Earning per share (accumulated value) Unit: Thousand Baht","Operating cash flow Unit: Thousand Baht","Investing cash flow Unit: Thousand Baht","Financing cash flow Unit: Thousand Baht"
            ,"Return on equity","Return on asset","Net profit margin (value of that quarter)","Net profit margin (accumulated value)","Debt to equity ratio","Fixed asset turnover ratio","Total asset turnover ratio"]
        }
    fin_data_dict_df = pd.DataFrame(fin_dict)
    fin_data_dict_text = convert_df_to_text(fin_data_dict_df)
    fin_data_dict_text = "df_fin DataFrame:\n" + fin_data_dict_text
    return fin_data_dict_text

def generate_recommend(df):
    fin_text = convert_df_to_text(df)
    fin_dict = create_financial_dict()
    fin_text = "df_fin DataFrame:\n" + fin_text
    prompt = f'''Act to financial consultant then analytic for this situation of this company
                          , here is the financial data of this company {fin_text}
                          , here is data dictionaries for this data {fin_dict}
                          , output from existing data or calculate from existing data only
                          , output with simple word only
                          , topic of output are Financial Performance, Valuation, Profitability, Liquidity, Overall and Recommend for investor
                          , output in plain text
                          '''
    response = model.generate_content(prompt, generation_config=genai.types.GenerationConfig(temperature=0.5))
    return response

def generate_price(df_quo):
    df_quo['date'] = pd.to_datetime(df_quo['date'])
    df_quo = df_quo.resample('W-SUN', on='date') \
        .agg({'prior': 'mean'
              , 'open': 'mean'
              , 'high': 'mean'
              , 'low': 'mean'
              , 'close': 'mean'
              , 'average': 'mean'
              , 'aomVolume': 'mean'
              , 'aomValue': 'mean'
              , 'trVolume': 'mean'
              , 'trValue': 'mean'
              , 'totalVolume': 'mean'
              , 'totalValue': 'mean'
              , 'pe': 'mean'
              , 'pbv': 'mean'
              , 'bvps': 'mean'
              , 'dividendYield': 'mean'
              , 'marketCap': 'mean'
              , 'volumeTurnover': 'mean'})
    df_quo = df_quo.reset_index()
    quote_dict = create_quote_dict()
    quo_text = convert_df_to_text(df_quo)
    quo_text = "df_quo DataFrame:\n" + quo_text
    prompt = f'''In role of data analytics in financial or banking business then provide or forecast direction(increase, decrease, same) of price for this company with period of next 1 day, next 1 week, next 1 month and next 1 quarter
                                  , here is the historical data of this company with price or detail in market {quo_text}
                                  , here is data dictionaries for this data {quote_dict}
                                  , output from existing data or calculate from existing data only
                                  , output is forecasting from this data
                                  , output are direction of each period
                                  , output in format **Direction** : explain with reason of your think
                                  , sample output **Next 1 day: Increase**, reason...
                                  , output in plain text
                                  '''
    response = model.generate_content(prompt, generation_config=genai.types.GenerationConfig(temperature=0.7))
    return response

def generate_table(df):
    llm = GooglePalm(api_key="Replace This With Your API Key")  # Replace with your actual API key
    sdf = SmartDataframe(df, config={"llm": llm})

    st.subheader("Balance Sheet")
    balance_sheet = generate_financial_metrics(df, "balance sheet", sdf)
    st.write(balance_sheet)

    st.subheader("Profit and Loss Statement")
    profit_loss_statement = generate_financial_metrics(df, "profit and loss statement", sdf)
    st.write(profit_loss_statement)

    st.subheader("Cash Flow Statement")
    cash_flow_statement = generate_financial_metrics(df, "cash flow statement", sdf)
    st.write(cash_flow_statement)

def generate_financial_metrics(df, table_type, sdf):
    fin_dict = create_financial_dict()

    if table_type == "balance sheet":
        prompt = f"Generate a summary of the balance sheet with Asset, Liabilities, and Shareholder's equity from the following data: {df.to_string()}"

    elif table_type == "profit and loss statement":
        prompt = f"Generate a summary of the profit and loss statement with profit, net profit, and opex from the following data: {df.to_string()}"

    elif table_type == "cash flow statement":
        prompt = f"Generate a summary of the cash flow statement from the following data: {df.to_string()}"

    response = sdf.chat(prompt)
    return response

def generate_plot(df, graph_type, x_column, y_column, color, auto_size, symbol):
    df['Year_Quarter'] = df['year'].astype(str) + ' Q' + df['quarter'].astype(str)
    llm = GooglePalm(api_key="Replace This With Your API Key")
    sdf = SmartDataframe(df, config={"llm": llm})
    prompt = (
        f"Generate a {graph_type} plot to visualize the financial data for {symbol}. "
        f"The X-axis should represent {x_column}, and the Y-axis should represent the {aggregation_type} of {y_column}. "
        f"Use the color {color} for the plot. The graph should be auto-sized based on the data to ensure it fits well. "
    )
    response = sdf.chat(prompt)
    fig, ax = plt.subplots(figsize=(6, 4))
    fig, ax = plt.subplots()
    if aggregation_type == 'sum':
        aggregated_df = df.groupby(x_column)[y_column].sum().reset_index()
    else:
        aggregated_df = df.groupby(x_column)[y_column].mean().reset_index()
    aggregated_df[y_column] = aggregated_df[y_column] / 10**6
    if graph_type == 'bar':
        bars = ax.bar(aggregated_df[x_column], aggregated_df[y_column], color=color)
        for bar in bars:
            yval = bar.get_height()
            ax.text(bar.get_x() + bar.get_width() / 2, yval, f'{yval:,.0f}', ha='center', va='bottom')
    elif graph_type == 'line':
        ax.plot(aggregated_df[x_column], aggregated_df[y_column], color=color, marker='o')
        for i, row in aggregated_df.iterrows():
            ax.text(row[x_column], row[y_column], f'{row[y_column]:,.0f}', ha='center', va='bottom')
    elif graph_type == 'scatter':
        ax.scatter(aggregated_df[x_column], aggregated_df[y_column], color=color)
        for i, row in aggregated_df.iterrows():
            ax.text(row[x_column], row[y_column], f'{row[y_column]:,.0f}', ha='center', va='bottom')
    plt.xlabel(x_column, fontsize=10)
    plt.ylabel(f'{aggregation_type.capitalize()} of {y_column} (in MB)', fontsize=10)
    plt.title(f'{graph_type.capitalize()} Plot of {y_column} by {x_column}', fontsize=10)
    plt.xticks(rotation=45, fontsize=10)
    plt.yticks(fontsize=10)
    plt.tight_layout()
    output_path = '/content/exports/charts/temp_chart.png'
    plt.savefig(output_path)
    plt.close()
    return output_path

def get_gemini_insights(company_name, model):
    text_to_query = (
        f"Act as a financial analyst and provide a detailed financial analysis of {company_name}, "
        f"which is a leading company in Thailand. The analysis should include the following sections:\n\n"
        f"1. Company Overview:\n"
        f"   - Provide general background information about {company_name}\n"
        f"2. Vision and Mission:\n"
        f"   - State the company's vision and mission statements\n\n"
        f"3. Market Position:\n"
        f"   - Describe the company's position in the Thai market and its competitive landscape\n\n"
        f"4. Recent Developments:\n"
        f"   - Outline any recent major developments, mergers, acquisitions, or strategic initiatives\n\n"
        f"5. Future Outlook:\n"
        f"   - Give an outlook on the company's future, including growth opportunities and potential challenges\n"
    )
    response = model.generate_content(text_to_query)
    return response.text

def create_pdf(content):
    pdf = FPDF(orientation='L')  # Set the page orientation to 'L' for landscape (wide page)
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)

    # Add content to the PDF with text wrapping and bold formatting for topics
    line_height = 10
    page_width = pdf.w - 2 * pdf.l_margin

    for line in content.split('\n'):
        if line.startswith('#'):
            line = line.lstrip('#').strip()  # Remove leading '#' and any leading/trailing whitespace
            pdf.set_font("Arial", size=12, style='B')
            pdf.multi_cell(page_width, line_height, txt=line, border=0, align='L')
            pdf.set_font("Arial", size=12)  # Reset to normal font
        elif line.startswith('* **') and line.endswith('**'):
            line = line.strip('* **').strip()  # Remove leading '* **' and trailing '**', and any leading/trailing whitespace
            pdf.set_font("Arial", size=12, style='B')
            pdf.multi_cell(page_width, line_height, txt=line, border=0, align='L')
            pdf.set_font("Arial", size=12)  # Reset to normal font
        elif line.startswith('**') and line.endswith('**'):
            line = line.strip('**').strip()  # Remove leading and trailing '**', and any leading/trailing whitespace
            pdf.set_font("Arial", size=12, style='B')
            pdf.multi_cell(page_width, line_height, txt=line, border=0, align='L')
            pdf.set_font("Arial", size=12)  # Reset to normal font
        else:
            line = line.replace('*', '')  # Remove any remaining '*' from the line
            pdf.multi_cell(page_width, line_height, txt=line, border=0, align='L')

    # Output the PDF to a file
    pdf_file = "financial_analysis.pdf"
    pdf.output(pdf_file)

    return pdf_file

if st.sidebar.button('Submit'):
    df = get_fin_data(symbol.upper(), start_year, start_quarter, end_year, end_quarter)
    df_quo = get_quote_data(symbol.upper(), start_year, start_quarter, end_year, end_quarter)
    if df.empty or df_quo.empty:
        st.write("No data available for the specified parameters.")
    else:
        model = connect_gemini()
        st.header(f'Financial analysis of {symbol} by :blue[Guru] :orange[Finance] :money_with_wings:')
        st.markdown("""
All content on this website, it is provided for informational and educational purposes only. Guru Finance does not endorse and disclaims any liability for this website.
""")
        st.write(df)
        # col1, col2, col3 = st.columns((3, 3, 3))
        col1, col3 = st.columns((4.5, 4.5))

        with col1:
            st.image(generate_plot(df, graph_type, x_column, y_column, color, auto_size, symbol))
            st.subheader("Company Insights", divider='gray')
            company_insights = get_gemini_insights(symbol, model)
            st.markdown(company_insights)

        # with col2:
        #     st.subheader(f"Current situation of {symbol}", divider='gray')
        #     recommendation = generate_recommend(df).text
        #     st.markdown(recommendation)
        #     st.subheader(f"{symbol} stock price direction", divider='gray')
        #     price_direction = generate_price(df_quo).text
        #     st.markdown(price_direction)

        with col3:
            generate_table(df)

        st.subheader(f"Current situation of {symbol}", divider='gray')
        recommendation = generate_recommend(df).text
        st.markdown(recommendation)
        st.subheader(f"{symbol} stock price direction", divider='gray')
        price_direction = generate_price(df_quo).text
        st.markdown(price_direction)


        pdf_content = f"#Financial analysis of {symbol}\n\n" + \
                      f"##Company Insights\n" + ('-'*150) + '\n' + company_insights + "\n\n" + \
                      f"##Current situation of {symbol}\n" + ('-'*150) + '\n' + recommendation + "\n\n" + \
                      f"##{symbol} stock price direction\n" + ('-'*150) + '\n' + price_direction
        pdf_file = create_pdf(pdf_content)

        with open(pdf_file, "rb") as file:
            btn = st.sidebar.download_button(
                label="Download Analysis as PDF",
                data=file,
                file_name=pdf_file,
                mime="application/pdf"
            )



Writing UI.py


In [ ]:
! wget -q -O - ipv4.icanhazip.com

35.189.186.173


In [ ]:
! streamlit run UI.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.189.186.173:8501

npx: installed 22 in 4.678s
your url is: https://smooth-donkeys-bow.loca.lt
